# Dataset Management: Split & Merge

Hier können wir den Datensatz aufteilen und wieder zusammenfügen.

**Funktionen:**
* **SPLIT:** Nimmt die Bilder aus `data/[Klasse]` und verteilt sie zufällig auf:
  * `data/train` (70%)
  * `data/val` (15%)
  * `data/test` (15%)
* **MERGE:** Macht das rückgängig. Schiebt alles zurück in `data/[Klasse]`, damit du z.B. neue Bilder hinzufügen und neu splitten kannst.

**Wichtig:** Der Ordner `data/all` wird dabei komplett ignoriert und bleibt unberührt.

In [1]:
import os
import shutil
import random
from pathlib import Path

# --- KONFIGURATION ---
SOURCE_DIR = Path("dataset/data")

OUTPUT_DIR = Path("dataset/")

IGNORED_FOLDERS = ["all", "train", "val", "test", ".ipynb_checkpoints", ".DS_Store"]

def split_data(train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
    """
    Nimmt Daten aus SOURCE_DIR und verteilt sie nach OUTPUT_DIR (in train/val/test).
    """

    # Sicherheitscheck: Schauen wir im OUTPUT Ordner nach, ob schon was da ist
    if (OUTPUT_DIR / "train").exists():
        print(f"⚠️  Im Zielordner '{OUTPUT_DIR}' existiert schon ein Split!")
        print("Bitte führe erst 'merge_data()' aus oder lösche den Zielordner.")
        return

    # 1. Klassen im QUELL-Ordner finden
    if not SOURCE_DIR.exists():
        print(f"❌ Quellordner '{SOURCE_DIR}' existiert nicht.")
        return

    classes = [d for d in SOURCE_DIR.iterdir() if d.is_dir() and d.name not in IGNORED_FOLDERS]

    if not classes:
        print(f"❌ Keine Klassen-Ordner in '{SOURCE_DIR}' gefunden.")
        return

    print(f"Quelle: {SOURCE_DIR}")
    print(f"Ziel:   {OUTPUT_DIR}")
    print(f"Klassen: {[c.name for c in classes]}")
    print(f"Verhältnis: {train_ratio*100}% / {val_ratio*100}% / {test_ratio*100}%\n")

    for class_dir in classes:
        class_name = class_dir.name

        # Alle Bilder holen
        images = [f for f in class_dir.glob("*") if f.is_file()]
        random.shuffle(images)

        # Anzahl berechnen
        n = len(images)
        n_train = int(n * train_ratio)
        n_val = int(n * val_ratio)

        train_imgs = images[:n_train]
        val_imgs = images[n_train:n_train + n_val]
        test_imgs = images[n_train + n_val:]

        # Helper Funktion zum Verschieben
        def move_to_output(file_list, target_split):
            # Zielpfad: OUTPUT_DIR / train / klasse
            target_dir = OUTPUT_DIR / target_split / class_name
            target_dir.mkdir(parents=True, exist_ok=True)

            for img in file_list:
                # Wir verschieben (move) von Source nach Output
                shutil.move(str(img), str(target_dir / img.name))

        move_to_output(train_imgs, "train")
        move_to_output(val_imgs, "val")
        move_to_output(test_imgs, "test")

        print(f"  -> {class_name}: {len(train_imgs)} Train, {len(val_imgs)} Val, {len(test_imgs)} Test")

        # Optional: Leeren Quell-Ordner löschen?
        # Besser nicht, falls noch versteckte Dateien drin sind.

    print(f"\n✅ SPLIT ABGESCHLOSSEN. Daten liegen jetzt in '{OUTPUT_DIR}'.")


def merge_data():
    """
    Holt alle Daten aus OUTPUT_DIR (train/val/test) zurück in SOURCE_DIR.
    """

    splits = ["train", "val", "test"]

    # Prüfen ob im Output Ordner was ist
    if not any((OUTPUT_DIR / s).exists() for s in splits):
        print(f"⚠️  Nichts zu mergen in '{OUTPUT_DIR}'.")
        return

    print(f"Starte Merge von '{OUTPUT_DIR}' zurück nach '{SOURCE_DIR}'...")

    for split in splits:
        split_path = OUTPUT_DIR / split

        if split_path.exists():
            # Gehe durch die Klassen im Split-Ordner (z.B. output/train/cup)
            for class_dir in split_path.iterdir():
                if class_dir.is_dir():
                    class_name = class_dir.name

                    # Ziel ist wieder der SOURCE DIR (z.B. data/cup)
                    target_dir = SOURCE_DIR / class_name
                    target_dir.mkdir(parents=True, exist_ok=True)

                    # Dateien verschieben
                    files = list(class_dir.glob("*"))
                    for f in files:
                        shutil.move(str(f), str(target_dir / f.name))

            # Leeren Split-Ordner löschen
            shutil.rmtree(split_path)
            print(f"  -> '{split}' aufgelöst und zurückgeschoben.")

    # Optional: Wenn der Output Ordner leer ist, den auch löschen
    if OUTPUT_DIR.exists() and not any(OUTPUT_DIR.iterdir()):
        OUTPUT_DIR.rmdir()

    print(f"\n✅ MERGE ABGESCHLOSSEN. Alle Bilder sind wieder im Original-Ordner '{SOURCE_DIR}'.")

In [2]:
# Führe dies aus, um die Ordner zu erstellen (Train/Val/Test)
split_data()

Gefundene Klassen zum Splitten: ['cup', 'pen', 'bottle', 'keyboard']
Split-Verhältnis: 70.0% / 15.0% / 15.0%

  -> cup: 70 Train, 15 Val, 15 Test
  -> pen: 70 Train, 15 Val, 15 Test
  -> bottle: 70 Train, 15 Val, 15 Test
  -> keyboard: 70 Train, 15 Val, 16 Test

✅ SPLIT ABGESCHLOSSEN. Ordnerstruktur ist jetzt bereit für Training.


In [ ]:
# Führe dies aus, um alles rückgängig zu machen (alles zurück in data/Klasse)
merge_data()